In [2]:
import pandas as pd
import numpy as np
import os
import pydicom
import matplotlib.pyplot as plt

from scipy.misc import imsave
from imageio import imwrite, imread
from tqdm import tqdm
from skimage import *
import itertools
from math import sqrt
from keras import backend as K
from keras.callbacks import History 
import gc

%matplotlib inline


import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.optimizers import SGD, Adam,Adagrad
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from math import sqrt
from keras.callbacks import History 
from keras.optimizers import Adam, SGD
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image
from keras.applications import xception
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,GlobalAvgPool2D, GlobalMaxPool2D
from keras.layers import Activation, Dropout, Flatten, Dense,Input,concatenate
from keras.layers import BatchNormalization
from keras.models import Model
from keras.activations import relu
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import ELU


from keras.applications.resnet50 import preprocess_input, resnet50


In [3]:
%run '/home/santhosr/Documents/Codes/utils.py'

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,rotation_range=50,horizontal_flip=True,validation_split=0.1)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [5]:
batchSize = 10
imageSize = (1024,1024)

### Generator

In [ ]:
class Generator(keras.utils.Sequence):
    
    def __init__(self,numImages,class0Folder, class1Folder):
        
        self.numImages = numImages
        self.class0Folder = class0Folder
        self.class1Folder = class1Folder
        
        
    def __len__(self):
        
        return 30
    
    def __getitem__(self,index):
        
        pairs, targets = self.getBatch(batch_size,posRatio=0.5, shuffleSubjects=True)
        
        return pairs, targets
        

     
        

    def getBatch(self,batchSize, posRatio = 0.5, shuffleSubjects = True):
    
        
        posSamples = int(batchSize*posRatio)
        negSamples = batchSize - posSamples
        

        #positiveSamples 
        posPairList = []
            
        imgClass = choice(10)  #Select class out of 10 possible classes

        for i in range(posSamples):
            
            if shuffleSubjects==True:
                imgClass = choice(10)

            imgChoice = choice(self.numImages, 2, replace = False)

            img1 = self.dataMap[imgClass][imgChoice[0]]
            img2 = self.dataMap[imgClass][imgChoice[1]]

            posPairList.append(((img1,img2),1))
                

        #negativeSamples
        negPairList = []

        if shuffleSubjects==True:
            
            for i in range(negSamples):
                #Selecting the classes for the negative pair
                imgClasses = list(choice(10, 2,replace=False))
                
                #Selecting the specific image from each image class
                img1Choice = choice(self.numImages, 1, replace = False)
                img2Choice = choice(self.numImages, 1, replace = False)
                
                img1 = self.dataMap[imgClasses[0]][img1Choice[0]]
                img2 = self.dataMap[imgClasses[1]][img2Choice[0]]
                
                negPairList.append(((img1,img2),0))

        pairList = posPairList + negPairList
    

        inputAimgs = np.array([x[0][0] for x in pairList])
        inputBimgs = np.array([x[0][1] for x in pairList])
        target = np.array([x[1] for x in pairList])        

        return( [inputAimgs,inputBimgs], target)
    
    def on_epoch_end(self):
        'Updates to be done after each epoch'
        a = 5
        
        
    def generate(self, batch_size, s="train"):
        """a generator for batches, so model.fit_generator can be used. """
        while True:
            pairs, targets = self.getBatch(batch_size,posRatio=0.5, shuffleSubjects=True)
            yield (pairs, targets)